In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import logging

from conda_forge_tick.utils import setup_logging

setup_logging("DEBUG")

In [ ]:
from conda_forge_tick.make_graph import make_graph
from conda_forge_tick.all_feedstocks import get_all_feedstocks
from conda_forge_tick.utils import load_graph

names = []

gx = load_graph(reset_bad=False)

gx = make_graph(names, gx, debug=True)

## Check Some Version Stuff

In [ ]:
from conda_forge_tick.update_upstream_versions import update_upstream_versions

In [ ]:
import copy

gxc = copy.deepcopy(gx)

for node in list(gxc.nodes):
    if node not in ["tzdata", "openssl", "jpeg", "cddlib"]:
        gxc.remove_node(node)

In [ ]:
update_upstream_versions(gxc, debug=True)

## Look at a Migration

In [ ]:
from conda_forge_tick.auto_tick import migration_factory, add_rebuild_broken_migrator

In [ ]:
mgs = []
add_rebuild_broken_migrator(mgs, gx)

In [ ]:
mg = mgs[0]

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)

In [ ]:
mmctx.effective_graph.nodes

In [ ]:
mg_name = "libffi33"

mgs = []
migration_factory(mgs, gx, only_keep=[mg_name])

for i in range(len(mgs)):
    if mgs[i].name == mg_name:
        break
mg = mgs[i]

In [ ]:
import copy

attrs = copy.deepcopy(mg.graph.nodes["python"]["payload"].data)

attrs["branch"] = "3.6"
mg.filter(attrs)

In [ ]:
import os
from conda_forge_tick.contexts import MigratorContext, MigratorSessionContext


mctx = MigratorSessionContext(
    circle_build_url=os.getenv("CIRCLE_BUILD_URL", ""),
    graph=gx,
    smithy_version="",
    pinning_version="",
    github_username="",
    github_password="",
    github_token="",
    dry_run=False,
)

mmctx = MigratorContext(session=mctx, migrator=mg)
mg.bind_to_ctx(mmctx)


In [ ]:
mmctx.effective_graph.nodes

## Check the Status Report

In [ ]:
from conda_forge_tick.status_report import  graph_migrator_status

In [ ]:
out2, build_sequence, gv = graph_migrator_status(mg, mg.graph)

In [ ]:
gv.view()

In [ ]:
build_sequence

In [ ]:
len(build_sequence)

In [ ]:
gx.nodes["proj"]["payload"].data